<a href="https://colab.research.google.com/github/KirpaDmitriy/FreeTimeExtractor/blob/main/FreeTimeExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Извлечение не занятых событиями календаря временных отрезков</h1>

In [1]:
from datetime import datetime, date, time, timedelta

MIN_DATETME = datetime(1999, 1, 1, 1, 1)

class UserOccupiedSegment:  # события календаря
  def __init__(self, event_id: str, start: datetime, finish: datetime):
    self.event_id = event_id
    self.from_datetime = start
    self.to_datetime = finish
  
  def __repr__(self):
    return f"Segment {self.event_id} from {self.from_datetime} till {self.to_datetime}"

Алгоритм следующий: идём по сортированному списку событий календаря. Для каждого текущего просматриваем, какие уже начавшиеся события успели также и закончится перед началом рассматриваемого. Среди них выбираем максимальный конец <i>PrevMAX</i> - время, начиная с которого между предыдущими событиями и текущим нет ни одного события. Отрезок [<i>PrevMAX</i>; <i>начало текущего события</i>] - отрезок свободного времени. События, которые кончились перед текущим удаляем из рассмотрения.

In [2]:
def get_free_time_segments_list(busy: list):
  busy.sort(key=lambda uos: uos.from_datetime)
  print(f"Busy sorted: {busy}")
  busy_dict = {}
  free_segments = []
  for uos in busy:
    current_start = uos.from_datetime
    busy_dict[uos.event_id] = uos.to_datetime

    just_before_current = MIN_DATETME
    just_before_current_valid = False
    for previous_id, previous_finish in list(busy_dict.items()):
      if previous_finish < current_start:
        just_before_current = max(just_before_current, previous_finish)
        just_before_current_valid = True
        busy_dict.pop(previous_id)
    if just_before_current_valid:
      free_segments.append({
          "start": just_before_current,
          "finish": current_start
      })
  
  return free_segments

In [3]:
busy_list = [
    UserOccupiedSegment("1", datetime(2022, 11, 20, 9, 0), datetime(2022, 11, 20, 17, 39)),
    UserOccupiedSegment("2", datetime(2022, 11, 20, 2, 4), datetime(2022, 11, 20, 2, 6)),
    UserOccupiedSegment("3", datetime(2022, 11, 20, 7, 8), datetime(2022, 11, 20, 8, 7)),
    UserOccupiedSegment("4", datetime(2022, 11, 20, 21, 20), datetime(2022, 11, 20, 23, 55)),
    UserOccupiedSegment("5", datetime(2022, 11, 20, 0, 1), datetime(2022, 11, 20, 1, 56)),
    UserOccupiedSegment("6", datetime(2022, 11, 20, 17, 40), datetime(2022, 11, 20, 18, 26)),
]
print(f"Busy raw: {busy_list}")

free_list = list(map(
    lambda fts: f"Time segment from {fts['start']} till {fts['finish']}",
    get_free_time_segments_list(busy_list)
))
print(f"Free: ", end=""); print(*free_list, sep=", ")

Busy raw: [Segment 1 from 2022-11-20 09:00:00 till 2022-11-20 17:39:00, Segment 2 from 2022-11-20 02:04:00 till 2022-11-20 02:06:00, Segment 3 from 2022-11-20 07:08:00 till 2022-11-20 08:07:00, Segment 4 from 2022-11-20 21:20:00 till 2022-11-20 23:55:00, Segment 5 from 2022-11-20 00:01:00 till 2022-11-20 01:56:00, Segment 6 from 2022-11-20 17:40:00 till 2022-11-20 18:26:00]
Busy sorted: [Segment 5 from 2022-11-20 00:01:00 till 2022-11-20 01:56:00, Segment 2 from 2022-11-20 02:04:00 till 2022-11-20 02:06:00, Segment 3 from 2022-11-20 07:08:00 till 2022-11-20 08:07:00, Segment 1 from 2022-11-20 09:00:00 till 2022-11-20 17:39:00, Segment 6 from 2022-11-20 17:40:00 till 2022-11-20 18:26:00, Segment 4 from 2022-11-20 21:20:00 till 2022-11-20 23:55:00]
Free: Time segment from 2022-11-20 01:56:00 till 2022-11-20 02:04:00, Time segment from 2022-11-20 02:06:00 till 2022-11-20 07:08:00, Time segment from 2022-11-20 08:07:00 till 2022-11-20 09:00:00, Time segment from 2022-11-20 17:39:00 till 20

Теперь можно достать отрезки свободного времени для заданного промежутка.

In [14]:
def get_free_time_segments_list_bounded(busy: list, lb: datetime, ub: datetime):
  fts = get_free_time_segments_list(busy)  # тут from_datetime посортированы
  fts = [
      ft for ft in fts
         if (lb <= ft["start"] <= ub) and (lb <= ft["finish"] <= ub)
  ]
  earliest_start = busy[0].from_datetime
  if earliest_start > lb:
    fts = [ {"start": lb, "finish": earliest_start} ] + fts
  latest_finish = max([bt.to_datetime for bt in busy])
  if latest_finish < ub:
    fts += [ {"start": latest_finish, "finish": ub} ]
  return fts

In [16]:
free_list_bounded = list(map(
    lambda fts: f"Time segment from {fts['start']} till {fts['finish']}",
    get_free_time_segments_list_bounded(
        busy_list,
        datetime(2022, 11, 20, 17, 40),
        datetime(2022, 11, 20, 23, 45)
)))
print(f"Free: ", end=""); print(*free_list_bounded, sep=", ")

Busy sorted: [Segment 5 from 2022-11-20 00:01:00 till 2022-11-20 01:56:00, Segment 2 from 2022-11-20 02:04:00 till 2022-11-20 02:06:00, Segment 3 from 2022-11-20 07:08:00 till 2022-11-20 08:07:00, Segment 1 from 2022-11-20 09:00:00 till 2022-11-20 17:39:00, Segment 6 from 2022-11-20 17:40:00 till 2022-11-20 18:26:00, Segment 4 from 2022-11-20 21:20:00 till 2022-11-20 23:55:00]
Free: Time segment from 2022-11-20 18:26:00 till 2022-11-20 21:20:00


In [17]:
free_list_extended = list(map(
    lambda fts: f"Time segment from {fts['start']} till {fts['finish']}",
    get_free_time_segments_list_bounded(
        busy_list,
        datetime(2022, 11, 19, 17, 40),
        datetime(2022, 11, 21, 23, 45)
)))
print(f"Free: ", end=""); print(*free_list_extended, sep=", ")

Busy sorted: [Segment 5 from 2022-11-20 00:01:00 till 2022-11-20 01:56:00, Segment 2 from 2022-11-20 02:04:00 till 2022-11-20 02:06:00, Segment 3 from 2022-11-20 07:08:00 till 2022-11-20 08:07:00, Segment 1 from 2022-11-20 09:00:00 till 2022-11-20 17:39:00, Segment 6 from 2022-11-20 17:40:00 till 2022-11-20 18:26:00, Segment 4 from 2022-11-20 21:20:00 till 2022-11-20 23:55:00]
Free: Time segment from 2022-11-19 17:40:00 till 2022-11-20 00:01:00, Time segment from 2022-11-20 01:56:00 till 2022-11-20 02:04:00, Time segment from 2022-11-20 02:06:00 till 2022-11-20 07:08:00, Time segment from 2022-11-20 08:07:00 till 2022-11-20 09:00:00, Time segment from 2022-11-20 17:39:00 till 2022-11-20 17:40:00, Time segment from 2022-11-20 18:26:00 till 2022-11-20 21:20:00, Time segment from 2022-11-20 23:55:00 till 2022-11-21 23:45:00
